This files is for training/generating HMMs with **three states**.

In [1]:
from hmmlearn import hmm
import pandas as pd
import os
import numpy as np
import pickle

In [2]:
"""
Appends the error values of the file(sequence) to X and the length of the sequence to the array lengths. 

@param filename: name of file/sequence
@param X: array of error values for all practice opportunities of one practice mode
@param lengths: array of lengths of the individual sequences
"""
def get_sequence(filename, X, lengths):
    file = pd.read_csv(filename)
    for i, row in file.iterrows():
        X.append(list(row[4:]))
    lengths.append(len(file))
    return X, lengths

In [3]:
"""
Tries to find the best HMM, by returning the model with the highest score/log probability. Note that this is
not necessarily the most reasonable, most-fitting model to the data, so exploring other models is important!

@param filename: practice mode

@return modelBest: model with the highest logProb
@return logProbMax: the highest logProb
@return PBest: the transition matrix corresponding to the best model
"""
def findBestHMM(filename, X, lengths):
    logProbMax = 0
    modelBest = None
    PBest = None
    for i in range(10):
        try:
            model, mus, sigmas, P, logProb, convergence = fitHMM(X, lengths)
            if logProb > logProbMax and convergence.converged == True: #wrong do not use -> all not just highest score
                logProbMax = logProb
                modelBest = model
                PBest = P 
        except:
            continue
    with open(filename + ".pkl", "wb") as file: pickle.dump(modelBest, file)
    return modelBest, logProbMax, PBest

In [4]:
"""
Generates a Hidden Markov Model. The startprobability and initial transition probability should 
be varied between retries.

@return model:
@return mus: means of states
@return sigmas: covariance matrix
@return P: transition probability
@return logProb:score of model
@return convergence: monitors convergence of model
"""
def fitHMM(X, lengths):
    # fit Gaussian HMM to Q
    model = hmm.GaussianHMM(n_components=3, n_iter=1000, init_params="cm")
    
    model.startprob_ = np.array([0.0, 1.0, 0.0]) # same initial state
    model.transmat_ = np.array([[0.8, 0.2, 0.0], # initial prob matrix, important is 0 in (1,3) (3,1) (3,2)
                                [0.05, 0.8, 0.15],
                                [0.0, 0.0, 1.0]])
     
    model.fit(X, lengths)
    # classify each observation as state 0 or 1
    #hidden_states = model.predict(X)
 
    # find parameters of Gaussian HMM
    convergence = model.monitor_
    mus = np.array(model.means_)
    sigmas = np.array(np.sqrt(np.array([np.diag(model.covars_[0]),np.diag(model.covars_[1])])))
    P = np.array(model.transmat_)
 
    # find log-likelihood of Gaussian HMM
    logProb = model.score(X, lengths)
 
    return model, mus, sigmas, P, logProb, convergence #hidden_states

In [5]:
"""
Main: Generates models for one practice mode (directory) or iteratively for all (don't forget to save them
in that case).
"""
#for directory in ["None", "right hand", "single note", "slower", "split hands", "both hands"]:
directory = "single note"
lengths = []
X = []
for filename in os.listdir(directory):
    X, lengths = get_sequence(directory + "/" + filename, X, lengths)
model, mus, sigmas, P, logProb, convergence = fitHMM(X, lengths) #directory equals practice mode
#model, logProb, P = findBestHMM(directory)
print("/n", directory)
print("P", P)
print("logProb", logProb)
#print("means", mus)
print("convergences", convergence)
print("")
#with open("testmodel.pkl", "wb") as file: pickle.dump(model, file)

/n single note
P [[1.00000000e+00 4.09542409e-32 0.00000000e+00]
 [1.66666666e-01 8.20512821e-01 1.28205128e-02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
logProb 6731.9710201843145
convergences ConvergenceMonitor(
    history=[1955.105902295426, 6185.09824157095, 6731.971019215117, 6731.971020184315],
    iter=4,
    n_iter=1000,
    tol=0.01,
    verbose=False,
)



In [6]:
"""For saving models:"""
#with open("identity5.pkl", "wb") as file: pickle.dump(model, file)

'For saving models:'

In [11]:
"""
Calculates and prints the state with the min/max error value(observation/performance errors) according
to the raw errors and another error metric, which adds 1 to each state if its the min/max error value in 
that error section(eg pitch).
This is then used to normalise the model. That means, that the transition matrix is saved in a way, that
column/row 0 is always the unmastered-disengaged state (high error), column/row 1 is the unmastered-engaged
state and column/row 2 is always the mastered state (low error).
"""
directory = "left hand"
with open("BestThreeStates/" + directory + "/" + directory + "4.pkl", "rb") as file:
            model = pickle.load(file)
print("Transition matrix")
print(model.transmat_)

#print("Model means")
#print(model.means_)
mus = model.means_

state0 = mus[0][5] + mus[0][11]
state1 = mus[1][5] + mus[1][11]
state2 = mus[2][5] + mus[2][11]

print([state0, state1, state2])

max_error = np.argmax([state0, state1, state2])
min_error = np.argmin([state0, state1, state2])

new_max, new_min = new_error_comp(mus)

print("state with min_error:", min_error)
print("state with max error:", max_error)

print("state with new min_error:", new_min)
print("state with new max error:", new_max)
#means
#covariance
#transition
indizes = []

"""
if new_min != 2:
    if new_min == 0:
        indizes = [2,1,0]
        model.transmat_ = swap_row(swap_column(model.transmat_, indizes), indizes)
        model.means_ = swap_row(model.means_, indizes)
    elif new_min == 1:
        print("Wow, hier läuft was schief! Das Model sollte raus")
"""
indizes = [2,1,0]
model.transmat_ = swap_row(swap_column(model.transmat_, indizes), indizes)
model.means_ = swap_row(model.means_, indizes)

print("Transition matrix after")
print(model.transmat_)

        
#with open("left hand4_new.pkl", "wb") as file: pickle.dump(model, file)


Transition matrix
[[1.00000000e+00 0.00000000e+00 0.00000000e+00]
 [6.25025781e-02 7.49995673e-01 1.87501749e-01]
 [0.00000000e+00 1.00000000e+00 7.52262820e-33]]
[1.1825672926018316, 1.0598242347352753, 1.9520265746498267]
min -1 -2 3
state with min_error: 1
state with max error: 2
state with new min_error: 1
state with new max error: 2
Transition matrix after
[[7.52262820e-33 1.00000000e+00 0.00000000e+00]
 [1.87501749e-01 7.49995673e-01 6.25025781e-02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [8]:
"""
Calculates another min/max state in regards to errors, as described above.
"""
def new_error_comp(mus):
    state0 = state1 = state2 = 0
    for i in range(len(mus[0])):
        emax = np.argmax([mus[0][i], mus[1][i], mus[2][i]])
        emin = np.argmin([mus[0][i], mus[1][i], mus[2][i]])
        if emax == 0:
            state0 += 1
        elif emax == 1:
            state1 += 1
        elif emax == 2:
            state2 += 1
        if emin == 0:
            state0 -= 1
        elif emin == 1:
            state1 -= 1
        elif emin == 2:
            state2 -= 1
    new_max = np.argmax([state0, state1, state2])
    new_min = np.argmin([state0, state1, state2])
    print("min", state0, state1, state2)
    return new_max, new_min

In [9]:
def swap_column(arr, indizes):
    arr[:, [0,1,2]] = arr[:, indizes]
    return arr

In [10]:
def swap_row(arr, indizes):
    arr[[0,1,2]] = arr[indizes]
    return arr

In [ ]:
"""
if new_min == 0 and new_max == 2:
    # 0 -> 2, 2-> 0
    indizes = [2,1,0]
elif new_min == 0 and new_max == 1:
    # 0 -> 2, 1-> 0, 2 -> 1
    indizes = [2,0,1]
elif new_min == 1 and new_max == 2:
    # 0 -> 1, 1 -> 2, 2 -> 0
    indizes = [1,2,0]
elif new_min == 1 and new_max == 0:
    # 1 -> 2, 2 -> 1
    indizes = [0,2,1]
elif new_min == 2 and new_max == 1:
    # 1 -> 0, 0 -> 1
    indizes = [1,0,2]
else:
    indizes = [0,1,2]
model.transmat_ = swap_row(swap_column(model.transmat_, indizes), indizes)
model.means_ = swap_row(model.means_, indizes)
#model.covars_ = swap_row(model.covars_, indizes)

print(model.transmat_)
#wit